In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'e:\\bengla text summarization\\train-pegasus-model-on-bengali-text-summarization-using-mlops'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_dir : Path
    local_data_file : Path
    unzip_dir : Path
    


In [4]:
from src.benglasummarization.constants import *
from src.benglasummarization.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_dir=config.source_dir,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config

In [10]:
import os
import zipfile
from src.benglasummarization.logging import logger
from tqdm.notebook import tqdm
from dataclasses import replace

class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
        
    def load_file(self):
        if os.path.exists(self.config.source_dir):
            self.config = replace(self.config, local_data_file = self.config.source_dir)
            logger.info(f'File found at: {self.config.local_data_file}')
        else:
            logger.info(f'File not found at: {self.config.source_dir}')
            raise FileNotFoundError(f'No file found at: {self.config.source_dir}')
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            total_files = len(zip_ref.infolist())
            for file in tqdm(iterable=zip_ref.infolist(), total=total_files, desc = 'Extracting Files'):
                zip_ref.extract(member = file, path = unzip_path)
                
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")
            
            
            
            
    
    

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-16 02:22:48,187: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-16 02:22:48,189: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-16 02:22:48,192: INFO: common: created directory at: artifacts]
[2024-10-16 02:22:48,196: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-16 02:22:48,198: INFO: 2796563959: File found at: E:\\bengla text summarization\BanSum.zip]


Extracting Files:   0%|          | 0/1 [00:00<?, ?it/s]

[2024-10-16 02:23:01,006: INFO: 2796563959: Extracted E:\\bengla text summarization\BanSum.zip to artifacts/data_ingestion]
